In [8]:
######################### This code executes the data labelling
# import the python mysql client
import pymysql
import pandas as pd
# Create a connection object to Win10 MySQL Server  %%%% Make sure XAMMPP is running Apache & MySQL start%%%
ipOfHost       = "10.1.1.12" 
dbUser          = "root"
dbUserPassword  = "admin"
dbName          = "network_data"
dbCharset       = "utf8mb4"
cursorType      = pymysql.cursors.DictCursor
 
databaseConnection   = pymysql.connect(host=ipOfHost,
                                       user=dbUser,
                                       password=dbUserPassword,
                                       db=dbName,
                                       charset=dbCharset,
                                       cursorclass=cursorType,
                                       autocommit=True)                              
try:
    # Cursor object creation
    cursorObject    = databaseConnection.cursor()                                  
    updateStatement1 = "update network_data.tb_data set label = 'attack' where saddr='10.1.1.131' or daddr='10.1.1.131';"
    updateStatement2 = "update network_data.tb_data set label = 'normal' where label != 'attack'  or label is null; "
    # Execute the SQL UPDATE statement
    cursorObject.execute(updateStatement1)
    cursorObject.execute(updateStatement2)
    # sqlSelectUpdated  = "select * from network_data.tb_data;
    sqlSelectUpdated = "select stime,ltime,dur,mean,sum,min,max,spkts,dpkts,sbytes,dbytes,srate,drate,label from network_data.tb_data;"
    # Execute the SQL SELECT query
    cursorObject.execute(sqlSelectUpdated)
    # Fetch the updated row
    updatedRow = cursorObject.fetchall()
    data= pd.DataFrame (updatedRow)
except Exception as e:
    print("Exeception occured:{}".format(e))
finally:
    databaseConnection.close()

In [10]:
data.head(5)

,stime,ltime,dur,mean,sum,min,max,spkts,dpkts,sbytes,dbytes,srate,drate,label
0,1.611758e+09,1.611758e+09,60.069996,60.069996,60.069996,60.069996,60.069996,3015,4688,237713,1046335,50.174801,78.025642,normal
1,1.611758e+09,1.611758e+09,0.000000,0.000000,0.000000,0.000000,0.000000,1,0,60,0,0.000000,0.000000,normal
2,1.611758e+09,1.611758e+09,38.189991,38.189991,38.189991,38.189991,38.189991,2,0,164,0,0.026185,0.000000,normal
3,1.611758e+09,1.611758e+09,60.011093,60.011093,60.011093,60.011093,60.011093,53,0,4876,0,0.866506,0.000000,normal
4,1.611758e+09,1.611758e+09,0.000000,0.000000,0.000000,0.000000,0.000000,1,0,60,0,0.000000,0.000000,normal


In [16]:
# handling missing values
df_zeros= data.fillna(0)
df_zeros[df_zeros.isnull().any(axis=1)].head()
df_zeros.head(5)

,stime,ltime,dur,mean,sum,min,max,spkts,dpkts,sbytes,dbytes,srate,drate,label
0,1.611758e+09,1.611758e+09,60.069996,60.069996,60.069996,60.069996,60.069996,3015,4688,237713,1046335,50.174801,78.025642,normal
1,1.611758e+09,1.611758e+09,0.000000,0.000000,0.000000,0.000000,0.000000,1,0,60,0,0.000000,0.000000,normal
2,1.611758e+09,1.611758e+09,38.189991,38.189991,38.189991,38.189991,38.189991,2,0,164,0,0.026185,0.000000,normal
3,1.611758e+09,1.611758e+09,60.011093,60.011093,60.011093,60.011093,60.011093,53,0,4876,0,0.866506,0.000000,normal
4,1.611758e+09,1.611758e+09,0.000000,0.000000,0.000000,0.000000,0.000000,1,0,60,0,0.000000,0.000000,normal


In [17]:
# Use Decsion Tree to classify normal and attack data with one-hot coding features 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score , confusion_matrix
from sklearn . tree import DecisionTreeClassifier


y = df_zeros['label']. values
X = df_zeros. drop ('label', axis =1) . values

# divide data into training and testing sets
X_train , X_test , y_train , y_test = train_test_split (X, y, test_size =0.3, random_state =1)

# select model
model = DecisionTreeClassifier ()

# train the model
model.fit ( X_train , y_train )

# prediction using the testing phase
y_pred = model.predict ( X_test )

# Measuring performance using Accuracy 
print ("Accuracy", accuracy_score (y_pred , y_test ))

#Measuring the performance using Confusion Matrix
print ("Confusion Matrix", confusion_matrix (y_pred , y_test ))

Accuracy 0.9937876680574873
Confusion Matrix [[7741    3]
 [  64 2977]]
